# First steps for Google Places API

**To Do**

- seit wann Gemini Modell

Meine Aufgaben
- Datensatz vorbereiten: Tübingen
- zwei Datensätze: Name und Place_id + alle weiteren infos


## Preparations

In [ ]:
#!pip install -r requirements.txt

In [7]:
import requests
from urllib.parse import urlencode
import googlemaps
import os
from dotenv import load_dotenv
import time
import pandas as pd

In [8]:
# import the api_key from the api_key.py file
from api_key import api_key

## 1. nearbysearch

In [ ]:
def get_place_info(address):
# Base URL
  base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# Parameters in a dictionary
  params = {
   "input": address,
   "inputtype": "textquery",
   "fields": "name,geometry/location,place_id",
   "key": api_key,
  }
# Send request and capture response
  response = requests.get(base_url, params=params)
# Check if the request was successful
  if response.status_code == 200:
    return response.json()
  else:
    return None

In [ ]:
info = get_place_info("tübingen")	
print(info)

longitude =  info["candidates"][0].get("geometry", {}).get("location", {}).get("lng")
latitude = info["candidates"][0].get("geometry", {}).get("location", {}).get("lat")

In [61]:
# all fields that are possible and interesting for us
field_nearby_places = [
    # Fields that trigger the Nearby Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.attributions", "places.businessStatus", "places.containingPlaces", 
    "places.displayName", "places.formattedAddress", "places.googleMapsLinks", 
    "places.id", "places.location", "places.name", "places.plusCode", 
    "places.primaryType", "places.primaryTypeDisplayName", "places.pureServiceAreaBusiness", 
    "places.shortFormattedAddress", "places.subDestinations", "places.types", 
    "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Nearby Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Nearby Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]


def get_nearby_places(latitude, longitude, radius=500, max_results=10):
    """
    Calls the Google Places API to get nearby places within a specified radius.    
    
    Args:
        latitude (float): Latitude of the center of the search area.
        longitude (float): Longitude of the center of the search area.
        radius (float): Search radius in meters (default 500).
        max_results (int): Maximum number of results to return (default is 10, range 1 - 20 possible).
    
    Returns:
        dict: A dictionary containing the search results.
    """

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchNearby"
    # JSON request body
    body = {"includedTypes": ["restaurant"],  # Restrict to restaurants only
            "maxResultCount": max_results,  # Max number of results to return
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": latitude,
                               "longitude": longitude},
                    "radius": radius  # Radius for the search
                }
            },
            "languageCode": "en",  # Language code for the results
        }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName" 
        #"X-Goog-FieldMask": ",".join(field_nearby_places) 
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [62]:
result = get_nearby_places(latitude, longitude, max_results=15, radius=100)

## 2. textsearch
https://developers.google.com/maps/documentation/places/web-service/text-search#text-search-requests

filtering options:
- includePureServiceAreaBusinesses: filter for physical business location true or false
- locationRestriction or locationBias --> if omited API uses IP biasing by default; but locationBias parameter can be overridden if the textQuery 
    - rectangular Viewport or as a circle
- minRating
- openNow
- priceLevels
- rankPreference
- regionCode



In [74]:
# all fields that are possible and interesting for us
field_textsearch = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.attributions", "places.id", "places.name", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", 
    "places.plusCode", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.shortFormattedAddress", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.routingSummaries", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]



def get_places_textsearch(query, radius=None, max_results=10, page_token=None):
    """
    Calls the Google Places Text Search API to get places matching a query.
    
    Args:
        query (str): The search query string, e.g., "restaurants in New York".
        latitude (float): Latitude of the center of the search area (optional).
        longitude (float): Longitude of the center of the search area (optional).
        radius (float): Search radius in meters (optional, default 500).
        max_results (int): Maximum number of results to return per page (default is 10, range 1-20).

    Returns:
        dict: A dictionary containing the search results.
    """

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    # JSON request body
    body = {
        "includedType": "restaurant",  # Restrict to restaurants only
        "strictTypeFiltering": True,  # Only return results of the specified type
        "textQuery": query,
        "pageSize": max_results,  # Specify max results per page
        "pageToken": page_token,  # Token for the next page of results
        "languageCode": "en",  # Language code for the results
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName,nextPageToken" 
        #"X-Goog-FieldMask": ",".join(field_textsearch) 
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [75]:
res = get_places_textsearch("spicy vegetarian food in Nürnberg", max_results=5)
res

{'places': [{'name': 'places/ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'id': 'ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'displayName': {'text': 'My Hao Vietnamese Vegan Cuisine',
    'languageCode': 'en'}},
  {'name': 'places/ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'id': 'ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'displayName': {'text': 'Chay Vegan Kitchen - Nürnberg',
    'languageCode': 'de'}},
  {'name': 'places/ChIJGSyLd9BXn0cRggtocihIwdk',
   'id': 'ChIJGSyLd9BXn0cRggtocihIwdk',
   'displayName': {'text': 'Shiva Restaurant Nürnberg', 'languageCode': 'en'}},
  {'name': 'places/ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'id': 'ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'displayName': {'text': 'Kaschemme Nürnberg', 'languageCode': 'de'}},
  {'name': 'places/ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'id': 'ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'displayName': {'text': 'Veganel', 'languageCode': 'en'}}],
 'nextPageToken': 'AdDdOWqpo5z4eqSdyCb4sTViQpk_VCz-dWILo7i96TZ_YjiS21so41I776kOfFdmYUnPMGiwYHWk1LOoAlb399QHzQ45DppxGdCk5j_EBhatOzAlpxuIwbf-V7AUBE5f5HMy

In [77]:
token = res['nextPageToken']

In [78]:
res2 = get_places_textsearch("spicy vegetarian food in Nürnberg", max_results=5, page_token=token)
res

{'places': [{'name': 'places/ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'id': 'ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'displayName': {'text': 'My Hao Vietnamese Vegan Cuisine',
    'languageCode': 'en'}},
  {'name': 'places/ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'id': 'ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'displayName': {'text': 'Chay Vegan Kitchen - Nürnberg',
    'languageCode': 'de'}},
  {'name': 'places/ChIJGSyLd9BXn0cRggtocihIwdk',
   'id': 'ChIJGSyLd9BXn0cRggtocihIwdk',
   'displayName': {'text': 'Shiva Restaurant Nürnberg', 'languageCode': 'en'}},
  {'name': 'places/ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'id': 'ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'displayName': {'text': 'Kaschemme Nürnberg', 'languageCode': 'de'}},
  {'name': 'places/ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'id': 'ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'displayName': {'text': 'Veganel', 'languageCode': 'en'}}],
 'nextPageToken': 'AdDdOWqpo5z4eqSdyCb4sTViQpk_VCz-dWILo7i96TZ_YjiS21so41I776kOfFdmYUnPMGiwYHWk1LOoAlb399QHzQ45DppxGdCk5j_EBhatOzAlpxuIwbf-V7AUBE5f5HMy

### 1.1 keyword + location /nearbysearch

#### cleaning function

In [9]:
def tidy_nearby_result(info):
    # Define a function to tidy a single result
    def tidy_get_result(result):
        return {
            "name": result.get("name"),
            "place_id": result.get("place_id"),
            "address": result.get("vicinity"),
            "latitude": result.get("geometry", {}).get("location").get("lat"),
            "longitude": result.get("geometry", {}).get("location").get("lng"),
            "rating": result.get("rating"),
            "user_ratings_total": result.get("user_ratings_total"),
            "price_level": result.get("price_level"),
            "types": result.get("types"),
            "opening_hours": result.get("opening_hours"),
            #"opening_hours": result.get("opening_hours", {}).get("open_now"),
            "business_status": result.get("business_status"),
        }

    # Apply tidy_result to each entry in info["results"] and create a list of dictionaries
    tidy_results_list = [tidy_get_result(result) for result in info.get("results", [])]
    
    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(tidy_results_list)
    return df

#### 1.1 get 20 results - default

In [18]:
def nearby_search(keyword, lat, lng):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Parameters in a dictionary
    params = {
        "key": api_key,
        "location": f"{lat},{lng}", # must be specified as latitude,longitude
        "radius": 1500, # or rankby=distance
        "keyword": keyword, # can be the name of the restaurant or the type of restaurant
        # "field": "...", # does not work for nearby search
        "language": "en",
        "type": "restaurant" # not clear if it works
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
         # Pass the JSON response data to the tidy function
        info = response.json()
        return tidy_nearby_result(info)
    else:
          print("Error:", response.status_code)

In [19]:
info = nearby_search("restaurant", latitude, longitude)
info.head()

,name,place_id,address,latitude,longitude,rating,user_ratings_total,price_level,types,opening_hours,business_status
0,Gasthaus Bären,ChIJc-huVdT6mUcR81h3AxKXJVw,"Schmiedtorstraße 3, Tübingen",48.521547,9.052052,4.4,1365,2.0,"[restaurant, bar, food, point_of_interest, est...",None,OPERATIONAL
1,Die Wurstküche,ChIJZ9kADtP6mUcRF3IM1orJ8aI,"Am Lustnauer Tor 8, Tübingen",48.521737,9.057156,4.3,1742,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
2,Restaurant Hölderlinturm,ChIJx774JR_7mUcR3_uhwg9KT_c,"Bursagasse 4, Tübingen",48.519377,9.055690,5.0,76,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
3,Esszimmer,ChIJ5bNxBdP6mUcRoihNgmDzZl4,"Am Lustnauer Tor 4, Tübingen",48.521313,9.057686,4.7,963,NaN,"[restaurant, meal_takeaway, food, point_of_int...",{'open_now': True},OPERATIONAL
4,Pan's Asian Kitchen,ChIJF2Wg93H7mUcRfK_KN-0yTRI,"Hechinger Str. 67, Tübingen",48.509597,9.059956,4.8,137,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL


#### 1.2 60 results - extension

In [ ]:
def nearby_search_all_results(keyword, lat, lng):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Initial parameters without `pagetoken`
    params = {
        "key": api_key,
        "location": f"{lat},{lng}",  # Latitude and longitude
        "radius": 1500,  # Search radius in meters
        "keyword": keyword,  # Keyword for search
        "type": "restaurant",  # Restrict to restaurants
        "language": "en"
    }
    
    all_results = []  # To store all results across pages

    while True:
        # Send request and capture response
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            all_results.extend(data.get("results", []))  # Append current page results to list

            # Check if there's a next_page_token for additional results
            next_page_token = data.get("next_page_token")
            if next_page_token:
                # Update params with pagetoken for the next request
                params["pagetoken"] = next_page_token

                # Wait at least 2 seconds before making the next request
                time.sleep(2) # needed to avoid OVER_QUERY_LIMIT
            else:
                break  # No more pages, exit loop
        else:
            print("Error:", response.status_code)
            break

    # Use tidy_nearby_result to format the results as a DataFrame
    return tidy_nearby_result({"results": all_results})

In [39]:
# Example usage
place_info = nearby_search_all_results("restaurant", latitude, longitude)
place_info.head()

,name,place_id,address,latitude,longitude,rating,user_ratings_total,price_level,types,opening_hours,business_status
0,The little Kitchen,ChIJg0eJ3XD_nkcRdoN56jjk6gQ,"Am Viktualienmarkt 6, Ingolstadt",48.762952,11.426687,4.9,26,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
1,Gasthaus Daniel,ChIJ4zhE2lD-nkcR84vDBWMbvZQ,"Roseneckstraße 1, Ingolstadt",48.763165,11.422513,4.5,1878,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
2,Weissbräuhaus zum Herrnbräu,ChIJuTcbPVH-nkcR1bE-NTwKBqY,"Dollstraße 3, Ingolstadt",48.763372,11.423738,4.3,2341,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
3,Mooshäusl,ChIJWQWSa2H-nkcRfbj3PnReCyQ,"Mooshäuslweg, Ingolstadt",48.762589,11.401645,4.4,1045,2.0,"[restaurant, food, bar, point_of_interest, est...",{'open_now': True},OPERATIONAL
4,Sultanahmet Köftecisi Ingolstadt,ChIJ5Yndj4r_nkcRZ8vZl6qSuVg,"Friedrich-Ebert-Straße 37, Ingolstadt",48.771954,11.441074,4.3,136,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL


### only keywords & one result /findplacefromtext
- best to find a single, highly relevant place

Documentation: https://developers.google.com/maps/documentation/places/web-service/search-find-place 
Caution: Place Search requests and Place Details requests do not return the same fields. Place Search requests return a subset of the fields that are returned by Place Details requests. If the field you want is not returned by Place Search, you can use Place Search to get a place_id, then use that Place ID to make a Place Details request.

fields possibilities: https://developers.google.com/maps/documentation/places/web-service/place-data-fields#places-api-fields-support
interesting for our use case: 
    name,place_id,type,url,formatted_address,user_ratings_total,rating,price_level

maybe interesting:
    address_components,adr_address,business_status,geometry,geometry/viewpoint,geometry/location,photo,plus_code,utc_offset,vicinity,wheelchair_accessible_entrance,current_opening_hours,formatted_phone_number,international_phone_number,opening_hours,secondary_opening_hours,website, curbside_pickup,delivery, dine_in,editorial_summary,reservable,serves_beer,serves_breakfast,serves_brunch,serves_dinner, serves_lunch,serves_vegetarian_food,serves_wine,takeout

In [14]:
def find_place_from_text(input):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
    # Parameters in a dictionary
    params = {
        "input": input,  # Text string for search, such as a place name or address
        "inputtype": "textquery",
        "fields": "name,place_id,type,formatted_address,user_ratings_total,rating,price_level,geometry/location", 
        "key": api_key,
        "language": "en",
        "type": "restaurant"
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
         return response.json()
    else:
          print("Error:", response.status_code)

In [16]:
info = find_place_from_text("falafel ingolstadt")
info

{'candidates': [{'formatted_address': 'Unterer Graben 89, 85049 Ingolstadt, Germany',
   'geometry': {'location': {'lat': 48.7657222, 'lng': 11.4306434}},
   'name': 'Aladin Imbiss',
   'place_id': 'ChIJZ-z_tqX_nkcRcZJygKOFnYs',
   'price_level': 1,
   'rating': 4.6,
   'types': ['meal_takeaway',
    'restaurant',
    'food',
    'point_of_interest',
    'establishment'],
   'user_ratings_total': 346}],
 'status': 'OK'}

## 3. place details
https://developers.google.com/maps/documentation/places/web-service/place-details 


for later:  Generate Google Maps link
f"https://www.google.com/maps/place/?q=place_id:{place_id}"
& under 'googleMapsLinks'\'reviewsUri'

In [31]:
field_place_details = ["id", "displayName", "name", "addressComponents", "adrFormatAddress", "formattedAddress","location", 
                    "types", "accessibilityOptions", "businessStatus", "containingPlaces", "displayName", "googleMapsLinks", 
                    "primaryType", "primaryTypeDisplayName", "pureServiceAreaBusiness", "subDestinations", "currentOpeningHours", 
                    "nationalPhoneNumber", "priceLevel", "priceRange", "rating", "regularOpeningHours", "regularSecondaryOpeningHours", "userRatingCount", 
                    "websiteUri",
                    "allowsDogs", "curbsidePickup", "delivery", "dineIn", "editorialSummary", "evChargeOptions", "fuelOptions", 
                    "goodForChildren", "goodForGroups", "goodForWatchingSports", "liveMusic", "menuForChildren", "parkingOptions", 
                    "paymentOptions", "outdoorSeating", "reservable", "restroom", "reviews", "servesBeer", "servesBreakfast", 
                    "servesBrunch", "servesCocktails", "servesCoffee", "servesDessert", "servesDinner", "servesLunch", 
                    "servesVegetarianFood", "servesWine", "takeout"]

def get_place_details(place_id):
    """
    Calls the Google Places API to get details about a place.
    
    Args:
        place_id (str): A text query for the place.
    
    Returns:
        dict: The JSON response from the API.
    """

    # Base URL
    base_url = f"https://places.googleapis.com/v1/places/{place_id}"
    # Query parameters
    params = {
        "languageCode": "en"  # This is a query parameter, not a header
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(field_place_details) 
    }
    # Send request and capture response
    response = requests.get(base_url, headers=headers, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [35]:
details = get_place_details("ChIJGTtrv9P6mUcRtLRexsK4n1E")
details

{'name': 'places/ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'id': 'ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'types': ['italian_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'nationalPhoneNumber': '07071 25157',
 'formattedAddress': 'Bursagasse, 72070 Tübingen, Germany',
 'addressComponents': [{'longText': 'Bursagasse',
   'shortText': 'Bursagasse',
   'types': ['route'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['locality', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['administrative_area_level_3', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'TÜ',
   'types': ['administrative_area_level_2', 'political'],
   'languageCode': 'de'},
  {'longText': 'Baden-Württemberg',
   'shortText': 'BW',
   'types': ['administrative_area_level_1', 'political'],
   'languageCode': 'de'},
  {'longText': 'Germany',
   'shortText': 'DE',
   